# ОПРЕДЕЛЕНИЕ СЛУЧАЕВ ПАДЕНИЯ ЧЕЛОВЕКА С ПОМОЩЬЮ ТЕХНОЛОГИЙ КОМПЬЮТЕРНОГО ЗРЕНИЯ 

## Описание проекта

Проект **[Detect-fall](https://github.com/tickflag/Detect-fall)** представляет собой систему, разработанную для определения случаев падения человека с использованием технологий компьютерного зрения.

### Авторы

- Львов Станислав
- Дмитриева Настя
- Иванов Павел
- Порфирьев Кирилл

> Код проекта доступен на [GitHub](https://github.com/tickflag/Detect-fall)

In [ ]:
pip install ultralytics opencv-python catboost

# Settings

In [ ]:
import torch
settings = {
    'device': '0', #device to run on, i.e. cuda device='0/1/2/3' or device='cpu'
    'source': './03.mp4', #Source directory for videos
    'pose_model': './yolov8s-pose.pt' #Source directory for yolov8-pose model
}
if not torch.cuda.is_available():
    settings['device'] = 'cpu'

# DETECT-FALL

- Обработка изображений с помощью **[yolov8](https://docs.ultralytics.com)**

In [ ]:
from ultralytics import YOLO

model_pose = YOLO('./yolov8s-pose.pt')

def use_yolo(source):
    return model_pose.predict(
        source=source,
        save=False, #While debugging True, else False
        conf=0.5,
        vid_stride=1,
        device=settings['device'],
        show_labels=False,
        show_conf=False,
        verbose=False,
        show=False
    )

- Загрузка нашей модели

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.load_model('./cats-and-tea.json')

- Основной код

In [ ]:
from catboost import CatBoostClassifier
import cv2
import datetime
from tkinter import messagebox
import threading
import tkinter as tk


def process_frame(frame):
    cv2.imwrite('./input.jpg', frame)
    predict = check_pose('./input.jpg')

    return predict

def main():

    frames = []
    len_frames = 20

    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(settings['source'])
    
    if not vc.isOpened():
        print("Error: Couldn't open video source.")
        return 0
    
    while True:
        success, frame = vc.read()

        if not success:
            print("Error: Couldn't read frame.")
            break

        fallen_people = process_frame(frame)
        try:    
            fallen_people = max(fallen_people) // max(fallen_people)
        except Exception:
            fallen_people = 0
        frames.append(fallen_people)

        if len(frames) > len_frames:
            frames = frames[1::]
        

        if sum(frames) / len_frames >= 0.8: #TODO Попробовать различные коэффиценты в случае наличия времени на одном датасете и выбрать лучшее
            print('человек упал\n', '-' * 50)
            x = threading.Thread(target=alarm_message)
            x.start()
            
            frames = []
            #TODO Написать функции запуска предупреждения через ткинтер в отдельном потоке или ченить другое использовать
    vc.release()



def check_pose(frame):
    time = datetime.datetime.now()
    result = use_yolo(frame)
    keypoints = []
    for res in result:
        for element in res:
            data = [0.0, 0.0]
            box = element.boxes.cpu().numpy().xyxy[0]
            for keypoint in element.keypoints.cpu().numpy().xy[0]:
                data[0] += round(int(keypoint[0]) / int(box[-1]), 3)
                data[1] += round(int(keypoint[1]) / int(box[2]), 3)
            keypoints.append(data)
    
    #print(f'Разметка и подготовка - {datetime.datetime.now() - time}')

    out_list = []

    for input_data in keypoints:
        metric = model.predict(prediction_type='Probability', data=input_data)
        out_list.append(0 if metric[0] >= metric[1] else 1)
        #print('Вероятность падения: ',metric[1])
    #print(datetime.datetime.now() - time)
    return out_list

def alarm_message():
    messagebox.showwarning(title='Внимание', message='Обнаружен упавший человек!')

if __name__ == '__main__':
    main()